# Loadcase Generator

In [ ]:
data_file = "Datafiles/11 Loadcases.xlsx"

In [ ]:
import pandas as pd
import numpy as np
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.22.0")
db = lusas.database()

delete_all_previous = True
if delete_all_previous:
    db.deleteAllAnalyses()
    db.createAnalysisStructural("Analysis 1", False)
    db.deleteLoadsets("all") # remaining post-prosessing loadsest

Read the excel definitions

In [ ]:
df = pd.read_excel(data_file, sheet_name="Loadcases", usecols=range(0,4))
print(df.head())

In [ ]:
# Disable the interface to improve performance
if lusas.getMajorVersionNumber() >= 22 : lusas.enableUI(False)

In [ ]:
base_analysis = ''
for analysis in db.getAnalyses():
    if analysis.isBase():
        base_analysis = analysis.getName()
        break

We'll create a wrapper function that checks for the existance of a loadcase and increments the name

In [ ]:
def safe_create_loadcase(loadcase_name:str, analysis_name:str) -> 'IFLoadcase':
    if not db.existsLoadset(loadcase_name):
        return db.createLoadcase(loadcase_name, analysis_name)
    else:
        i = 1
        while db.existsLoadset(f"{loadcase_name}({i})"):
            i+=1
        db.createLoadcase(f"{loadcase_name} - {i}", analysis_name)

Loop throgh the rows in the loadcase table creating the loadcases as required

In [ ]:
for i, row in df.iterrows():
    loadcase_name = row['Name']
    loadcase_count = row['Count']
    # Make sure we have a string for comparison, it should be Yes or empty
    add_gravity = row['Gravity'] if isinstance(row['Gravity'], str) else ""
    # For the analysis name, we need a string and if the analysis doesnt exist we'll need to create it before trying to add the loadcase
    analysis_name = row['Analysis']

    if not isinstance(analysis_name, str):
        analysis_name = base_analysis
    else:
        if not db.existsAnalysis(analysis_name):
            db.createAnalysisStructural(analysis_name, False)

    # If the number of requested "similar" loadcases is less than two, create a single loadcase
    if np.isnan(loadcase_count) or int(loadcase_count) < 2:
        loadcase = safe_create_loadcase(loadcase_name, analysis_name)
        if add_gravity == "Yes": loadcase.addGravity(True)
    else:
        # Multiple loadcases
        for i in range(1, int(loadcase_count)+1):
            loadcase = safe_create_loadcase(f"{loadcase_name} {i}", analysis_name)
            if add_gravity == "Yes": loadcase.addGravity(True)


In [ ]:
def safe_create_envelope(envelope_name:str) -> 'IFEnvelope':
    if not db.existsLoadset(envelope_name):
        return db.createEnvelope(envelope_name)
    else:
        i = 1
        while db.existsLoadset(f"{envelope_name}({i})"):
            i+=1
        return db.createEnvelope(f"{envelope_name} - {i}")

Read in the envelope definition sheets and print the top for rows to confirm data as expected

In [ ]:
df = pd.read_excel(data_file, sheet_name="Envelopes", usecols=range(0,4))
print(df.head())

In [ ]:


for name in df['Name'].unique():
    #print(name)
    df_env = df[df['Name'] == name]
    #print(df_env)
    # Loadset ids to add to the envelope
    ids = []         
    for i, row in df_env.iterrows():
        loadcase_name = row['Loadcases']
        find_similar = row['FindSimilar']
        # Add all lodsets that start with the given name
        if find_similar == "Yes":
            for loadset in db.getLoadsets():
                if str(loadset.getName()).startswith(loadcase_name):
                    ids.append(loadset.getID())
        else:
            if db.existsLoadset(loadcase_name):
                loadset = db.getLoadset(loadcase_name)
                type_code = loadset.getTypeCode()
                # append the id
                ids.append(loadset.getID())
                if type_code == 3:
                    ids.append(win32.CastTo(loadset, 'IFEnvelope').getAssocLoadset().getID())
                elif type_code == 6:
                    ids.append(win32.CastTo(loadset, 'IFSmarCombination').getAssocLoadset().getID())

    
    env = safe_create_envelope(name)
    for id in ids:
        env.addEntry(id)

    # Set the treeview folder name if available
    folder_name = row['Folder']
    if isinstance(folder_name, str) and len(folder_name) > 0:
        env.setTreeLocation(folder_name, True)


In [ ]:
# Re-enable the interface. If something goes wrong above this cell must be manually run
if lusas.getMajorVersionNumber() >= 22 : lusas.enableUI(True)